In [227]:
import pandas as pd
import pickle
import re
from  sklearn.ensemble import RandomForestClassifier
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import resample
from datetime import *
from statistics import mean
from random import seed
from random import randint

In [391]:
df = pd.read_csv('chicago_data_frame.csv')
df = df.set_index(['hotel_id', 'reviewer_id'])

In [392]:
with open("reviews_file_chicago.txt", "rb") as fp:   # Unpickling
    review_count = pickle.load(fp)

In [312]:
regex = r'<hotelUrl>\n(.*?)\n</hotelUrl>'
unique_hotel_ids = set()
for i in review_count:
    hotel_name = re.search(regex, i)
    hotel_name = hotel_name[1]
    unique_hotel_ids.add(hotel_name)

In [313]:
def convert_to_date(date):
    ans = datetime.strptime(date, "%b %d, %Y")
    return ans

In [314]:
regex = r'<hotelUrl>\n(.*?)\n</hotelUrl>\n[\s\S]+?\n<helpfulness>\n(.*?)\n</helpfulness>'
unique_hotel_ids = set()
for i in review_count:
    hotel_name = re.search(regex, i)
    hotel_id = hotel_name[1]
    helpful_score = (hotel_name[2]).split(' of ')
    if (int(helpful_score[1])) > 4:
        unique_hotel_ids.add(hotel_id)


In [315]:
date_member_hotel = []
obj = []
count = 0
date_member_hotel_regex = r'<memberUrl>\n([\s\S]*?)\n</memberUrl>\n<hotelUrl>\n([\s\S]*?)\n</hotelUrl>\n[\s\S]*?\n<date>\n(.*?)\n</date>'
for i in review_count:
    matched = re.search(date_member_hotel_regex, i)
    member = matched[1]
    hotel = matched[2]
    date = matched[3]
    obj.append([hotel, member, date])

In [316]:
rows_with_hotel_as_key = dict()

for i in unique_hotel_ids:
    if i in df.index.get_level_values('hotel_id'):
        if i in rows_with_hotel_as_key:
            rows_with_hotel_as_key[i] = pd.merge(rows_with_hotel_as_key[i], df.loc[[i]], how = 'right', on=['hotel_id', 'reviewer_id'])
        else:
            rows_with_hotel_as_key[i] = pd.DataFrame(df.loc[[i]])

In [317]:
def removekey(d, key):
    r = dict(d)
    r.pop(key)
    return r

In [318]:
rows_with_hotel_as_key_culled_neg = dict()

for i in rows_with_hotel_as_key:
    if len(rows_with_hotel_as_key[i]) < 5:
        continue
    else:
        neg = 0
        for j,row in rows_with_hotel_as_key[i].iterrows():
            if (row['ST1']) < 4:
                neg+=1
        if neg > 4:
            rows_with_hotel_as_key_culled_neg[i] = rows_with_hotel_as_key[i]

In [319]:
len(rows_with_hotel_as_key_culled_neg)

103

In [320]:
rows_with_hotel_as_key_culled_pos = dict()

for i in rows_with_hotel_as_key:
    if len(rows_with_hotel_as_key[i]) < 5:
        continue
    else:
        pos = 0
        for j,row in rows_with_hotel_as_key[i].iterrows():
            if (row['ST1']) >= 4:
                pos+=1
        if pos > 4:
            rows_with_hotel_as_key_culled_pos[i] = rows_with_hotel_as_key[i]

In [321]:
len(rows_with_hotel_as_key_culled_pos)

298

In [322]:
x_train = pd.DataFrame(columns=df.columns)
x_train = pd.DataFrame(columns=df.columns)
x_train['hotel_id'] = ''
x_train['reviewer_id'] = ''
x_train = x_train.set_index(['hotel_id', 'reviewer_id'])

In [323]:
classifier = RandomForestClassifier()
hotel_prediction = dict()
y_pred = []
y_test = []
y_pred_df_neg = dict()
for i in rows_with_hotel_as_key_culled_neg:
    #create a dataframe I can use to train using leave one out approach
    # i is the hotel we want to use to test the classifier which is trained using the other hotels
    x_test = rows_with_hotel_as_key_culled_neg[i].copy()
    y_train = []
    x_train = pd.DataFrame(columns=df.columns)
    for j in rows_with_hotel_as_key_culled_neg:
        if i != j:
            new_df = rows_with_hotel_as_key_culled_neg[j].copy()
            x_train = pd.concat([x_train, new_df])
    y_train = x_train.pop('helpful_of_not')
    y_train = (y_train.to_frame()['helpful_of_not']).astype(str).astype(int)
    classifier.fit(X=x_train.values, y=y_train.values)
    y_test.append(x_test.pop('helpful_of_not'))
    prediction = classifier.predict_proba(x_test)
    y_pred_df_neg[x_test.index[0][0]] = pd.DataFrame(prediction, columns = ['not helpful', 'helpfulness'], index=x_test.index)


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [324]:
for i in y_pred_df_neg:
    y_pred_df_neg[i] = pd.merge(y_pred_df_neg[i], df, how = 'left', on=['hotel_id', 'reviewer_id'])

In [325]:
y_pred_fin_neg = dict()

for i in (y_pred_df_neg):
    count =0
    for j,row in y_pred_df_neg[i].iterrows():
        y_pred_fin_neg[i] = y_pred_df_neg[i][y_pred_df_neg[i]['ST1'] < 4]

In [326]:
for i in (y_pred_fin_neg):
    y_pred_fin_neg[i].sort_values(by=['helpfulness'], inplace=True, ascending=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [327]:
neg_class_avg = []

for i in (y_pred_fin_neg):
    top = y_pred_fin_neg[i].iloc[0]
    neg_class_avg.append((top['helpfulness']))

In [328]:
neg_class_total = []

for i in (y_pred_fin_neg):
    top = y_pred_fin_neg[i].iloc[0]
    helpful = (top['helpfulness'])
    if helpful >= 0.5:
        neg_class_total.append(1)
    else:
        neg_class_total.append(0)

In [329]:
mean(neg_class_avg)

0.8912621359223301

In [330]:
mean(neg_class_total)

0.9902912621359223

In [331]:
x_train_pos = pd.DataFrame(columns=df.columns)
x_train_pos = pd.DataFrame(columns=df.columns)
x_train_pos['hotel_id'] = ''
x_train_pos['reviewer_id'] = ''
x_train_pos = x_train_pos.set_index(['hotel_id', 'reviewer_id'])

In [332]:
classifier = RandomForestClassifier(n_estimators=39, max_depth=20)
hotel_prediction = dict()
y_pred = []
y_test = []
y_pred_df_pos = dict()
for i in rows_with_hotel_as_key_culled_pos:
    #create a dataframe I can use to train using leave one out approach
    # i is the hotel we want to use to test the classifier which is trained using the other hotels
    x_test = rows_with_hotel_as_key_culled_pos[i].copy()
    y_train = []
    x_train = pd.DataFrame(columns=df.columns)
    for j in rows_with_hotel_as_key_culled_pos:
        if i != j:
            new_df = rows_with_hotel_as_key_culled_pos[j].copy()
            x_train_pos = pd.concat([x_train, new_df])
    y_train_pos = x_train_pos.pop('helpful_of_not')
    y_train = (y_train_pos.to_frame()['helpful_of_not']).astype(str).astype(int)
    classifier.fit(X=x_train_pos.values, y=y_train.values)
    y_test.append(x_test.pop('helpful_of_not'))
    prediction = classifier.predict_proba(x_test)
    y_pred_df_pos[x_test.index[0][0]] = pd.DataFrame(prediction, columns = ['not helpful', 'helpfulness'], index=x_test.index)
    

In [333]:
for i in y_pred_df_pos:
    y_pred_df_pos[i] = pd.merge(y_pred_df_pos[i], df, how = 'left', on=['hotel_id', 'reviewer_id'])

In [334]:
y_pred_fin_pos = dict()

for i in (y_pred_df_pos):
    count =0
    for j,row in y_pred_df_pos[i].iterrows():
        y_pred_fin_pos[i] = y_pred_df_pos[i][y_pred_df_pos[i]['ST1'] >= 4]

In [335]:
for i in (y_pred_fin_pos):
    y_pred_fin_pos[i].sort_values(by=['helpfulness'], inplace=True, ascending=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [336]:
pos_class_avg = []

for i in (y_pred_fin_pos):
    top = y_pred_fin_pos[i].iloc[0]
    pos_class_avg.append((top['helpfulness']))

In [337]:
pos_class_total = []

for i in (y_pred_fin_pos):
    top = y_pred_fin_pos[i].iloc[0]
    helpful = (top['helpfulness'])
    if helpful >= 0.5:
        pos_class_total.append(1)
    else:
        pos_class_total.append(0)

In [338]:
mean(pos_class_avg)

0.6270005162622613

In [339]:
mean(pos_class_total)

0.697986577181208

# Date Negative Mean Review Helpfulness

In [340]:
for i in y_pred_fin_neg:
    y_pred_fin_neg[i]['date'] = ''

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [341]:
for j in y_pred_fin_neg:
    for i in obj: 
        test_tuple = tuple([i[0],i[1]])
        if test_tuple in y_pred_fin_neg[j].index:
            date = convert_to_date(i[2])
            y_pred_fin_neg[j].set_value(y_pred_fin_neg[j].loc[[test_tuple]].index,'date',  date)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/anaconda3/lib/python3.7/site-packages/ipykernel/zmqshell.py:536: PerformanceWarning: indexing past lexsort depth may impact performance.
  return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)


In [342]:
for i in (y_pred_fin_neg):
    y_pred_fin_neg[i].sort_values(by=['date'], inplace=True, ascending=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [343]:
hold = y_pred_fin_neg.copy()
for i in hold:
    if len(hold[i]) < 1:
        del y_pred_fin_neg[i]

In [344]:
values_helpful_of_not_neg = df['helpful_of_not']

In [345]:
for i in y_pred_fin_neg:
    y_pred_fin_neg[i] = pd.merge(y_pred_fin_neg[i], values_helpful_of_not_neg, how = 'left', on=['hotel_id', 'reviewer_id'])

In [346]:
for i in y_pred_fin_neg:
    y_pred_fin_neg[i] = pd.merge(y_pred_fin_neg[i], values_helpful_of_not_neg, how = 'left', on=['hotel_id', 'reviewer_id'])

In [347]:
for i in (y_pred_fin_neg):
    y_pred_fin_neg[i].sort_values(by=['date'], inplace=True, ascending=False)

In [348]:
total = []
for i in (y_pred_fin_neg):
    top = (y_pred_fin_neg[i].iloc[0])['helpful_of_not']
    total.append(top)

total_fin = list(map(int, total))

In [349]:
mean(total_fin)

0.34951456310679613

# Positive Date Mean Review Helpfulness

In [350]:
for i in y_pred_fin_pos:
    y_pred_fin_pos[i]['date'] = ''

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [351]:
for j in y_pred_fin_pos:
    for i in obj: 
        test_tuple = tuple([i[0],i[1]])
        if test_tuple in y_pred_fin_pos[j].index:
            date = convert_to_date(i[2])
            y_pred_fin_pos[j].set_value(y_pred_fin_pos[j].loc[[test_tuple]].index,'date',  date)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [352]:
for i in (y_pred_fin_pos):
    y_pred_fin_pos[i].sort_values(by=['date'], inplace=True, ascending=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [353]:
hold = y_pred_fin_pos.copy()
for i in hold:
    if len(hold[i]) < 1:
        del y_pred_fin_pos[i]

In [354]:
values_helpful_of_not_pos = df['helpful_of_not']

In [355]:
for i in y_pred_fin_pos:
    y_pred_fin_pos[i] = pd.merge(y_pred_fin_pos[i], values_helpful_of_not_pos, how = 'left', on=['hotel_id', 'reviewer_id'])

In [356]:
for i in y_pred_fin_pos:
    y_pred_fin_pos[i] = pd.merge(y_pred_fin_pos[i], values_helpful_of_not_pos, how = 'left', on=['hotel_id', 'reviewer_id'])

In [357]:
for i in (y_pred_fin_pos):
    y_pred_fin_pos[i].sort_values(by=['date'], inplace=True, ascending=False)

In [358]:
total = []
for i in (y_pred_fin_pos):
    top = (y_pred_fin_pos[i].iloc[0])['helpful_of_not']
    total.append(top)

total_fin = list(map(int, total))

In [359]:
mean(total_fin)

0.5134228187919463

# Random Negative Percentage of recommended reviews that are helpful

In [360]:
for i in y_pred_fin_neg:
    del y_pred_fin_neg[i]['date']

In [361]:
for i in y_pred_fin_neg:
    rand_int = randint(0,len(y_pred_fin_neg[i])-1)
    top = (y_pred_fin_neg[i].iloc[rand_int])['helpful_of_not']
    total.append(top)

In [362]:
total_fin_neg = list(map(int, total))

In [363]:
mean(total_fin_neg)

0.4912718204488778

In [364]:
df.pop('helpful_of_not')

hotel_id                                                                                                                       reviewer_id                                              
/Hotel_Review-g294212-d302012-Reviews-Grand_Hotel_Beijing-Beijing.html                                                         /members-reviews/UpperWestSiderGuy                           0
http://las-vegas-hotels.tripadvisor.com/Hotel_Review-g45963-d91816-Reviews-Four_Queens_Hotel_and_Casino-Las_Vegas_Nevada.html  /members-reviews/Canuckscout                                 0
/Hotel_Review-g37209-d111623-Reviews-Embassy_Suites_Hotel_Indianapolis_North-Indianapolis_Indiana.html                         /members-reviews/slags68                                     0
/Hotel_Review-g47042-d92830-Reviews-Royal_Holiday_Motel-Gallup_New_Mexico.html                                                 /members-reviews/notaheiress                                 0
/Hotel_Review-g187497-d501108-Reviews-AC_Barcelona-Barc

In [365]:
vals_df = pd.read_csv('vals_df.csv')

In [366]:
vals_df = vals_df.set_index(['hotel_id', 'reviewer_id'])

In [367]:
df = pd.merge(vals_df, df, how = 'left', on=['hotel_id', 'reviewer_id'])

In [368]:
rows_with_hotel_as_key = dict()

for i in unique_hotel_ids:
    if i in df.index.get_level_values('hotel_id'):
        if i in rows_with_hotel_as_key:
            rows_with_hotel_as_key[i] = pd.merge(rows_with_hotel_as_key[i], df.loc[[i]], how = 'right', on=['hotel_id', 'reviewer_id'])
        else:
            rows_with_hotel_as_key[i] = pd.DataFrame(df.loc[[i]])

In [369]:
rows_with_hotel_as_key_culled_neg = dict()

for i in rows_with_hotel_as_key:
    if len(rows_with_hotel_as_key[i]) < 5:
        continue
    else:
        neg = 0
        for j,row in rows_with_hotel_as_key[i].iterrows():
            if (row['ST1']) < 4:
                neg+=1
        if neg > 4:
            rows_with_hotel_as_key_culled_neg[i] = rows_with_hotel_as_key[i]

In [370]:
for i in y_pred_fin_neg:
    y_pred_fin_neg[i]['date'] = ''

In [371]:
for j in y_pred_fin_neg:
    for i in obj: 
        test_tuple = tuple([i[0],i[1]])
        if test_tuple in y_pred_fin_neg[j].index:
            date = convert_to_date(i[2])
            y_pred_fin_neg[j].set_value(y_pred_fin_neg[j].loc[[test_tuple]].index,'date',  date)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [372]:
for i in (y_pred_fin_neg):
    y_pred_fin_neg[i].sort_values(by=['date'], inplace=True, ascending=False)

In [373]:
total = []
for i in (y_pred_fin_neg):
    top = (y_pred_fin_neg[i].iloc[0])['helpfulness']
    total.append(top)

In [374]:
print(type(total))
total_fin = list(map(float, total))

<class 'list'>


In [375]:
mean(total_fin)

0.40194174757281553

# Random Positive Percentage of recommended reviews that are helpful

In [376]:
for i in y_pred_fin_pos:
    rand_int = randint(0,len(y_pred_fin_pos[i])-1)
    top = (y_pred_fin_pos[i].iloc[rand_int])['helpful_of_not']
    total.append(top)

In [377]:
total_fin_pos = list(map(int, total))

In [378]:
mean(total_fin_pos)

0.45885286783042395

In [379]:
df.pop('helpful_of_not')

hotel_id                                                                                                                reviewer_id                 
/Hotel_Review-g42881-d90408-Reviews-Embassy_Suites_Minneapolis_Airport-Bloomington_Minnesota.html                       /members-reviews/sungoddess3    1.000000
http://cancun-hotels.tripadvisor.com/Hotel_Review-g150807-d185322-Reviews-Riu_Caribe-Cancun_Yucatan_Peninsula.html      /members-reviews/sungoddess3    1.000000
/Hotel_Review-g33116-d81632-Reviews-Comfort_Inn_Suites_San_Francisco_Airport_North-South_San_Francisco_California.html  /members-reviews/sungoddess3    1.000000
/Hotel_Review-g274887-d276600-Reviews-Art_otel_budapest_by_park_plaza-Budapest.html                                     /members-reviews/europa00       0.000000
/Hotel_Review-g35790-d90819-Reviews-Fairfield_Inn_Champaign-Champaign_Illinois.html                                     /members-reviews/europa00       0.000000
                                              

In [380]:
vals_df = pd.read_csv('vals_df.csv')

In [381]:
vals_df = vals_df.set_index(['hotel_id', 'reviewer_id'])

In [382]:
df = pd.merge(vals_df, df, how = 'left', on=['hotel_id', 'reviewer_id'])

In [383]:
rows_with_hotel_as_key = dict()

for i in unique_hotel_ids:
    if i in df.index.get_level_values('hotel_id'):
        if i in rows_with_hotel_as_key:
            rows_with_hotel_as_key[i] = pd.merge(rows_with_hotel_as_key[i], df.loc[[i]], how = 'right', on=['hotel_id', 'reviewer_id'])
        else:
            rows_with_hotel_as_key[i] = pd.DataFrame(df.loc[[i]])

In [384]:
rows_with_hotel_as_key_culled_pos = dict()

for i in rows_with_hotel_as_key:
    if len(rows_with_hotel_as_key[i]) < 5:
        continue
    else:
        pos = 0
        for j,row in rows_with_hotel_as_key[i].iterrows():
            if (row['ST1']) >= 4:
                pos+=1
        if pos > 4:
            rows_with_hotel_as_key_culled_pos[i] = rows_with_hotel_as_key[i]

In [385]:
for i in y_pred_fin_pos:
    y_pred_fin_pos[i]['date'] = ''

In [386]:
for j in y_pred_fin_pos:
    for i in obj: 
        test_tuple = tuple([i[0],i[1]])
        if test_tuple in y_pred_fin_pos[j].index:
            date = convert_to_date(i[2])
            y_pred_fin_pos[j].set_value(y_pred_fin_pos[j].loc[[test_tuple]].index,'date',  date)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [387]:
for i in (y_pred_fin_pos):
    y_pred_fin_pos[i].sort_values(by=['date'], inplace=True, ascending=False)

In [388]:
total = []
for i in (y_pred_fin_pos):
    top = (y_pred_fin_pos[i].iloc[0])['helpfulness']
    total.append(top)

In [389]:
print(type(total))
total_fin = list(map(float, total))

<class 'list'>


In [390]:
mean(total_fin)

0.3614696265702977